# &#x1F4D1; &nbsp; $\mathfrak {\color{#348ABD} {P7: \ Building \ a \ Student \ Intervention \ System}}$

## $\mathfrak {\color{#348ABD} {1. \ Introduction}}$
### Question 1 - Classification vs. Regression
*The goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*
### Answer 1

## $\mathfrak {\color{#348ABD} {2. \ Code \ Library}}$

In [2]:
from IPython.core.display import HTML
hide_code = ''
HTML('''<script>code_show = true; 
function code_display() {
    if (code_show) {
        $('div.input').each(function(id) {
            if (id == 0 || $(this).html().indexOf('hide_code') > -1) {$(this).hide();}
        });
        $('div.output_prompt').css('opacity', 0);
    } else {
        $('div.input').each(function(id) {$(this).show();});
        $('div.output_prompt').css('opacity', 1);
    }
    code_show = !code_show;
} 
$(document).ready(code_display);</script>
<form action="javascript: code_display()"><input style="color: #348ABD; background: ghostwhite; opacity: 0.9; " \
type="submit" value="Click to display or hide code"></form>''')

In [3]:
hide_code
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

################################
### ADD EXTRA LIBRARIES HERE ###
################################


## $\mathfrak {\color{#348ABD} {3. \ Exploring \ the \ Data}}$
We will start from loading the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [33]:
hide_code
# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")
student_data.info()

Student data read successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 31 columns):
school        395 non-null object
sex           395 non-null object
age           395 non-null int64
address       395 non-null object
famsize       395 non-null object
Pstatus       395 non-null object
Medu          395 non-null int64
Fedu          395 non-null int64
Mjob          395 non-null object
Fjob          395 non-null object
reason        395 non-null object
guardian      395 non-null object
traveltime    395 non-null int64
studytime     395 non-null int64
failures      395 non-null int64
schoolsup     395 non-null object
famsup        395 non-null object
paid          395 non-null object
activities    395 non-null object
nursery       395 non-null object
higher        395 non-null object
internet      395 non-null object
romantic      395 non-null object
famrel        395 non-null int64
freetime      395 non-null int64
goout         395 non

Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. We will need to compute the following:

- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).

In [29]:
hide_code
# Calculate number of students
n_students = len(student_data)

# Calculate number of features
n_features = len(list(student_data.T.index))

# Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# Calculate graduation rate
grad_rate = n_passed * 100.0 / n_students

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## $\mathfrak {\color{#348ABD} {4. \ Preparing \ the \ Data}}$
In this section, we will prepare the data for modeling, training and testing.

### 4.1 Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

In [32]:
hide_code
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head().T)

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
                  0        1        2         3       4
school           GP       GP       GP        GP      GP
sex               F        F        F         F       F
age              18       17       15        15      16
address           U        U        U         U       U
famsize         GT3      GT3      LE3       GT3     GT3
Pstatus           A        T        T         T       T
Medu              4        1        1         4       3
Fedu              4        1        1         2       3
Mjob        at_home  at_home  at_home    health   other
Fjob        teacher    other    other  services   other
reason       cour